In [1]:
import pandas as pd

### Read Expected and Obesrved subtitutions and take only CytB and 3 Position

In [2]:
obs_mut = pd.read_csv('../data/ObsMutSpec.csv')
obs_mut = obs_mut[obs_mut['3Pos'] == 1]
obs_mut = obs_mut[obs_mut['Gene'] == 'CytB']
exp_mut = pd.read_csv('../data/exp_mut_spec.csv',sep='\t')
exp_mut = exp_mut[exp_mut['3Pos'] == 1]
exp_mut = exp_mut[exp_mut['Gene'] == 'CYTB']

### Check how many species we have in each data set

In [16]:
sps_exp = exp_mut['Species']
sps_obs = obs_mut['Species']
print(len(sps_exp.unique()), len(sps_obs.unique()), sep=', ')


1024, 1024


### Take species that presented in each dataset 

In [6]:
obs_mut = obs_mut.loc[obs_mut['Species'].isin(sps_exp.unique())]
exp_mut = exp_mut.loc[exp_mut['Species'].isin(sps_obs.unique())]

### Count codons in each gene for each species for observed and expected mutations

In [7]:
count_obs_df = obs_mut.groupby(['Species', 'Gene']).Mut3.value_counts()
count_obs_df.name = 'CountMut3'
count_obs_df = count_obs_df.reset_index()
final_obs = count_obs_df.pivot(index=['Species', 'Gene'], columns='Mut3', values='CountMut3').fillna(0)
final_obs['ExpectedObserved'] = 'Observed'

In [8]:
count_exp_df = exp_mut.groupby(['Species', 'Gene']).Mut3.value_counts()
count_exp_df.name = 'CountMut3'
count_exp_df = count_exp_df.reset_index()
final_exp = count_exp_df.pivot(index=['Species', 'Gene'], columns='Mut3', values='CountMut3').fillna(0)
final_exp['ExpectedObserved'] = 'Expected'

In [9]:
final_exp.head()

,Mut3,A[A>C]A,A[A>C]C,A[A>C]G,A[A>C]T,A[A>G]A,A[A>G]C,A[A>G]G,A[A>G]T,A[A>T]A,A[A>T]C,...,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T,ExpectedObserved
Species,Gene,,,,,,,,,,,,,,,,,,,,,
Abbottina_obtusirostris,CYTB,8.0,4.0,4.0,4.0,8.0,4.0,4.0,4.0,8.0,4.0,...,9.0,10.0,14.0,12.0,9.0,10.0,14.0,12.0,9.0,Expected
Abbottina_rivularis,CYTB,6.0,3.0,3.0,3.0,6.0,3.0,3.0,3.0,6.0,3.0,...,10.0,5.0,9.0,8.0,10.0,5.0,9.0,8.0,10.0,Expected
Abramis_brama,CYTB,9.0,3.0,4.0,5.0,9.0,3.0,4.0,5.0,9.0,3.0,...,9.0,10.0,15.0,13.0,9.0,10.0,15.0,13.0,9.0,Expected
Acanthogobius_hasta,CYTB,7.0,2.0,3.0,2.0,7.0,2.0,3.0,2.0,7.0,2.0,...,11.0,14.0,16.0,22.0,11.0,14.0,16.0,22.0,11.0,Expected
Acanthopagrus_latus,CYTB,8.0,3.0,4.0,3.0,8.0,3.0,4.0,3.0,8.0,3.0,...,14.0,11.0,12.0,18.0,14.0,11.0,12.0,18.0,14.0,Expected


### Merge tables and sort by species

In [10]:
final_df = pd.concat([final_exp, final_obs])
final_df = final_df.sort_values(['Species', 'Gene'])
final_df.insert(0, 'ExpectedObserved', final_df.pop('ExpectedObserved'))

In [15]:
final_df.to_csv('../data/counted_codons_cytb.csv')